In [1]:
import os
import glob

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
tf.random.set_seed(1234)


2024-06-21 04:29:01.304109: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 04:29:01.304202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 04:29:01.478767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
DATA_DIR = tf.keras.utils.get_file(
    "dataset2019.zip",
    "http://download.cs.stanford.edu/downloads/completion3d/dataset2019.zip",
    extract=True,
)
DATA_DIR = os.path.join (os.path.dirname(DATA_DIR),"shapenet")

1585860897/1585860897 ━━━━━━━━━━━━━━━━━━━━ 300s 0us/step


In [3]:
import re
import h5py
import numpy as np

def read_point_cloud_from_h5_file(file_path):
    with h5py.File(file_path, 'r') as file:
        
        point_cloud_data = file['data'][:]
    return point_cloud_data

In [21]:
import os
import h5py
import numpy as np
from tqdm import tqdm

def load_h5_file(file_path, dataset_name='data'):
    """Load point cloud data from an .h5 file."""
    with h5py.File(file_path, 'r') as file:
        data = file[dataset_name][:]
    return data

def load_dataset(base_dir, categories, dataset_name='data'):
    """Load partial and gt datasets from the given base directory."""
    partials = []
    gts = []

    for category in categories:
        partial_dir = os.path.join(base_dir, 'partial', category)
        gt_dir = os.path.join(base_dir, 'gt', category)

        # Check if the directories exist
        if not os.path.exists(partial_dir):
            print(f"Partial directory does not exist: {partial_dir}")
            continue
        if not os.path.exists(gt_dir):
            print(f"GT directory does not exist: {gt_dir}")
            continue

        partial_files = sorted([f for f in os.listdir(partial_dir) if f.endswith('.h5')])
        gt_files = sorted([f for f in os.listdir(gt_dir) if f.endswith('.h5')])

        for p_file, gt_file in tqdm(zip(partial_files, gt_files), total=len(partial_files), desc=f"Loading {category}"):
            partial_path = os.path.join(partial_dir, p_file)
            gt_path = os.path.join(gt_dir, gt_file)

            partial_data = load_h5_file(partial_path, dataset_name)
            gt_data = load_h5_file(gt_path, dataset_name)

            partials.append(partial_data)
            gts.append(gt_data)

    return np.array(partials), np.array(gts)

# Example usage
base_dir = os.path.join(DATA_DIR , "train")
categories = ['02691156', '03636649', '02933112', '02958343', '03001627' , '04256520' , '04379243' , '04530566']  # Replace with your actual category names
partial_dataset, gt_dataset = load_dataset(base_dir, categories)
print(f'Loaded {len(partial_dataset)} partial point clouds and {len(gt_dataset)} ground truth point clouds.')


Loading 04530566: 100%|██████████| 1689/1689 [00:01<00:00, 983.45it/s] 


Loaded 28974 partial point clouds and 28974 ground truth point clouds.
